In [1]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
%matplotlib inline

from sksurv.linear_model import CoxPHSurvivalAnalysis, CoxnetSurvivalAnalysis
from sksurv.preprocessing import OneHotEncoder
from sksurv.datasets import load_breast_cancer
from sksurv.nonparametric import kaplan_meier_estimator

from sklearn.model_selection import GridSearchCV, KFold
from sklearn.exceptions import ConvergenceWarning
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('clinical.csv')
df.head()

,Num,Patient,surv_status,surv_time,DF_surv_status,DF_surv_time,DM_status,DM_time,IM_status,IM_time,...,clinical_44,clinical_45,clinical_46,clinical_47,clinical_48,clinical_49,clinical_50,clinical_51,clinical_52,clinical_53
0,1,SICK0000022,0,20.866667,0,11.233333,1,11.233333,0,20.866667,...,1,0,3,0,60.0,1,0,253,417,0.606715
1,2,SICK0000027,1,74.666667,0,31.600000,0,74.666667,0,74.666667,...,0,0,3,0,80.0,0,0,151,261,0.578544
2,3,SICK0000053,1,58.633333,1,58.633333,0,58.633333,0,58.633333,...,1,1,2,0,60.0,1,0,178,349,0.510029
3,4,SICK0000085,1,58.533333,1,58.533333,0,58.533333,0,58.533333,...,0,0,3,1,70.0,0,1,236,248,0.951613
4,5,SICK0000213,0,40.433333,0,13.766667,1,13.766667,1,13.766667,...,1,0,3,1,50.0,0,1,223,277,0.805054


In [3]:
df["surv_status"]=df["surv_status"].astype(bool)
df["surv_status"]

0      False
1       True
2       True
3       True
4      False
       ...  
378     True
379     True
380     True
381     True
382    False
Name: surv_status, Length: 383, dtype: bool

In [4]:
df["DF_surv_status"]=df["DF_surv_status"].astype(bool)
df["DF_surv_status"]

0      False
1      False
2       True
3       True
4      False
       ...  
378     True
379     True
380     True
381     True
382    False
Name: DF_surv_status, Length: 383, dtype: bool

In [5]:
data=df.iloc[0:, 2:4]
data1=df.iloc[0:, 4:6]
X=df.iloc[0:, 10:]

y=data.to_records(index=False)
y1=data1.to_records(index=False)
X=OneHotEncoder().fit_transform(X)

In [6]:
y.shape

(383,)

In [7]:
X

,clinical_01,clinical_02,clinical_03,clinical_04,clinical_05,clinical_06,clinical_07,clinical_08,clinical_09,clinical_10,...,clinical_44,clinical_45,clinical_46,clinical_47,clinical_48,clinical_49,clinical_50,clinical_51,clinical_52,clinical_53
0,1,45,2,1,1,2,1,2,0,1,...,1,0,3,0,60.0,1,0,253,417,0.606715
1,1,39,1,1,0,1,0,1,0,1,...,0,0,3,0,80.0,0,0,151,261,0.578544
2,1,41,2,1,0,2,3,2,1,1,...,1,1,2,0,60.0,1,0,178,349,0.510029
3,1,41,2,1,0,2,0,2,0,1,...,0,0,3,1,70.0,0,1,236,248,0.951613
4,1,40,1,1,0,1,0,1,0,1,...,1,0,3,1,50.0,0,1,223,277,0.805054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,1,44,2,1,1,1,0,1,0,1,...,1,1,2,1,15.0,0,1,168,155,1.083871
379,1,39,1,1,0,2,0,2,0,1,...,0,0,2,2,25.0,0,0,199,358,0.555866
380,1,46,2,1,0,1,0,1,0,1,...,1,0,2,0,19.0,0,1,238,277,0.859206
381,1,37,1,1,0,2,0,2,0,1,...,1,1,3,1,35.0,0,0,211,501,0.421158


In [8]:
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
X_std = stdsc.fit_transform(X)
print(X_std.shape)

X1 = pd.DataFrame(X_std)

(383, 51)


In [9]:
X1

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,-0.432603,-0.421167,-0.075970,0.312124,2.043447,0.194922,0.301265,0.020139,-0.291523,0.126155,...,0.716822,-0.683691,0.435248,-0.922380,0.179393,1.489459,-1.186931,0.135217,0.420086,-0.557795
1,-0.432603,-0.982234,-1.530804,0.312124,-0.489369,-1.187579,-0.767110,-1.522536,-0.291523,0.126155,...,-1.395046,-0.683691,0.435248,-0.922380,1.106305,-0.671385,-1.186931,-1.234697,-0.754314,-0.705969
2,-0.432603,-0.795211,-0.075970,0.312124,-0.489369,0.194922,2.438014,0.020139,3.430258,0.126155,...,0.716822,1.462650,-2.297540,-0.922380,0.179393,1.489459,-1.186931,-0.872073,-0.091832,-1.066354
3,-0.432603,-0.795211,-0.075970,0.312124,-0.489369,0.194922,-0.767110,0.020139,-0.291523,0.126155,...,-1.395046,-0.683691,0.435248,0.390897,0.642849,-0.671385,0.842509,-0.093102,-0.852181,1.256337
4,-0.432603,-0.888723,-1.530804,0.312124,-0.489369,-1.187579,-0.767110,-1.522536,-0.291523,0.126155,...,0.716822,-0.683691,0.435248,0.390897,-0.284064,-0.671385,0.842509,-0.267699,-0.633863,0.485452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,-0.432603,-0.514678,-0.075970,0.312124,2.043447,-1.187579,-0.767110,-1.522536,-0.291523,0.126155,...,0.716822,1.462650,-2.297540,0.390897,-1.906160,-0.671385,0.842509,-1.006378,-1.552304,1.952002
379,-0.432603,-0.982234,-1.530804,0.312124,-0.489369,0.194922,-0.767110,0.020139,-0.291523,0.126155,...,-1.395046,-0.683691,-2.297540,1.704175,-1.442704,-0.671385,-1.186931,-0.590031,-0.024078,-0.825254
380,-0.432603,-0.327655,-0.075970,0.312124,-0.489369,-1.187579,-0.767110,-1.522536,-0.291523,0.126155,...,0.716822,-0.683691,-2.297540,-0.922380,-1.720778,-0.671385,0.842509,-0.066241,-0.633863,0.770285
381,-0.432603,-1.169256,-1.530804,0.312124,-0.489369,0.194922,-0.767110,0.020139,-0.291523,0.126155,...,0.716822,1.462650,0.435248,0.390897,-0.979248,-0.671385,-1.186931,-0.428865,1.052455,-1.533807


In [10]:
X_std

array([[-0.43260311, -0.4211666 , -0.07597043, ...,  0.13521701,
         0.42008562, -0.55779491],
       [-0.43260311, -0.98223376, -1.53080414, ..., -1.23469679,
        -0.75431432, -0.70596943],
       [-0.43260311, -0.79521138, -0.07597043, ..., -0.87207255,
        -0.0918323 , -1.066354  ],
       ...,
       [-0.43260311, -0.32765541, -0.07597043, ..., -0.0662409 ,
        -0.63386304,  0.77028464],
       [-0.43260311, -1.16925615, -1.53080414, ..., -0.42886515,
         1.05245481, -1.53380688],
       [-0.43260311,  1.35554607,  1.37886329, ...,  1.49170028,
         0.21682409,  1.02531921]])

In [11]:
CS_OS = -0.138842*X1[11] -0.028678*X1[15] + 0.078265*X1[19] +0.152441*X1[47]
CS_OS

0     -0.238635
1     -0.140467
2     -0.238635
3      0.070735
4      0.046953
         ...   
378    0.070735
379   -0.238635
380    0.240248
381   -0.116686
382   -0.000610
Length: 383, dtype: float64

In [13]:
CS_OS.to_excel('CS_OS.xlsx', index = False)

In [14]:
CS_DFS = -0.134233*X1[11] +0.072858*X1[19] -0.009350*X1[25] +0.157052*X1[47]
CS_DFS

0     -0.262497
1     -0.104695
2     -0.262497
3      0.056231
4      0.056231
         ...   
378    0.056231
379   -0.262497
380    0.214032
381   -0.104695
382    0.056231
Length: 383, dtype: float64

In [15]:
CS_DFS.to_excel('CS_DFS.xlsx', index = False)

In [16]:
df = pd.read_csv('radiomics.csv')
df.head()

,Num,Patient,surv_status,surv_time,DF_surv_status,DF_surv_time,DM_status,DM_time,IM_status,IM_time,...,original_gldm_LargeDependenceLowGrayLevelEmphasis,original_gldm_LowGrayLevelEmphasis,original_gldm_SmallDependenceEmphasis,original_gldm_SmallDependenceHighGrayLevelEmphasis,original_gldm_SmallDependenceLowGrayLevelEmphasis,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,1,SICK0000022,0,20.866667,0,11.233333,1,11.233333,0,20.866667,...,0.007047,0.005523,0.893044,723.780508,0.005183,0.021380,0.040092,2175.588315,0.162204,46.800219
1,2,SICK0000027,1,74.666667,0,31.600000,0,74.666667,0,74.666667,...,0.017158,0.013204,0.881673,375.646698,0.012330,0.034383,0.048880,890.006763,0.111293,33.735629
2,3,SICK0000053,1,58.633333,1,58.633333,0,58.633333,0,58.633333,...,0.005932,0.004502,0.892413,634.309034,0.004174,0.025478,0.034650,1651.539734,0.111554,35.270325
3,4,SICK0000085,1,58.533333,1,58.533333,0,58.533333,0,58.533333,...,0.007314,0.006214,0.907639,798.071759,0.005984,0.021264,0.034183,3366.444227,0.176277,53.006708
4,5,SICK0000213,0,40.433333,0,13.766667,1,13.766667,1,13.766667,...,0.032357,0.014786,0.829630,383.970267,0.010553,0.022626,0.071841,997.527090,0.081362,92.718598


In [17]:
df["surv_status"]=df["surv_status"].astype(bool)
df["surv_status"]

0      False
1       True
2       True
3       True
4      False
       ...  
378     True
379     True
380     True
381     True
382    False
Name: surv_status, Length: 383, dtype: bool

In [18]:
df["DF_surv_status"]=df["DF_surv_status"].astype(bool)
df["DF_surv_status"]

0      False
1      False
2       True
3       True
4      False
       ...  
378     True
379     True
380     True
381     True
382    False
Name: DF_surv_status, Length: 383, dtype: bool

In [19]:
data=df.iloc[0:, 2:4]
data1=df.iloc[0:, 4:6]
X=df.iloc[0:, 10:]

y=data.to_records(index=False)
y1=data1.to_records(index=False)
X=OneHotEncoder().fit_transform(X)

In [20]:
y.shape

(383,)

In [21]:
X

,original_shape_Elongation,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,original_shape_MinorAxisLength,original_shape_Sphericity,original_shape_SurfaceArea,...,original_gldm_LargeDependenceLowGrayLevelEmphasis,original_gldm_LowGrayLevelEmphasis,original_gldm_SmallDependenceEmphasis,original_gldm_SmallDependenceHighGrayLevelEmphasis,original_gldm_SmallDependenceLowGrayLevelEmphasis,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,0.652719,295.129395,224,135,227.903488,227.903488,129.500000,192.636689,0.123373,1003.330870,...,0.007047,0.005523,0.893044,723.780508,0.005183,0.021380,0.040092,2175.588315,0.162204,46.800219
1,0.635921,363.259177,272,160,275.697298,275.697298,157.833333,231.004258,0.117527,1201.747845,...,0.017158,0.013204,0.881673,375.646698,0.012330,0.034383,0.048880,890.006763,0.111293,33.735629
2,0.503933,490.105753,359,184,365.242385,365.242385,167.166667,246.980671,0.115311,1272.660114,...,0.005932,0.004502,0.892413,634.309034,0.004174,0.025478,0.034650,1651.539734,0.111554,35.270325
3,0.762606,317.414357,248,176,258.938217,258.938217,135.000000,242.062199,0.124331,1023.592600,...,0.007314,0.006214,0.907639,798.071759,0.005984,0.021264,0.034183,3366.444227,0.176277,53.006708
4,0.830215,404.746523,279,231,295.040675,295.040675,153.166667,336.026565,0.120796,1146.056308,...,0.032357,0.014786,0.829630,383.970267,0.010553,0.022626,0.071841,997.527090,0.081362,92.718598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,0.717886,873.197578,624,407,638.330635,638.330635,397.750000,626.855971,0.091125,2870.246982,...,0.180166,0.067871,0.874989,1315.969437,0.046542,0.016625,0.023141,9541.175317,0.235757,112.517106
379,0.575893,517.326342,368,208,374.412874,374.412874,207.500000,297.924503,0.109352,1550.022879,...,0.018910,0.006984,0.808236,540.007382,0.004158,0.019909,0.046957,1227.041175,0.077056,62.009593
380,0.540729,495.886981,368,192,369.855377,369.855377,188.083333,268.140368,0.112701,1408.609103,...,0.003107,0.002588,0.919438,1916.831040,0.002466,0.014310,0.029738,4913.134056,0.288836,55.281928
381,0.577302,717.900851,512,278,517.176952,517.176952,314.750000,414.445459,0.094394,2370.559430,...,0.022673,0.009268,0.792582,558.711616,0.007185,0.021833,0.047224,1297.736483,0.077672,82.482901


In [22]:
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
X_std = stdsc.fit_transform(X)
print(X_std.shape)

X1 = pd.DataFrame(X_std)

(383, 103)


In [23]:
X1

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,100,101,102
0,-0.277646,-1.155660,-1.037620,-1.463721,-1.164129,-1.164129,-1.150015,-1.337368,1.121542,-1.144516,...,-0.310984,-0.355218,1.044680,-0.393412,-0.372741,0.069278,-0.089397,-0.006375,0.473809,-0.524082
1,-0.394298,-0.785080,-0.689893,-1.137081,-0.818383,-0.818383,-0.772031,-0.986286,0.681191,-0.791081,...,-0.246029,-0.105441,0.699667,-1.256776,0.112159,1.190792,0.563269,-0.924423,-0.390144,-0.912339
2,-1.310862,-0.095121,-0.059638,-0.823507,-0.170604,-0.170604,-0.647519,-0.840094,0.514312,-0.664767,...,-0.318148,-0.388423,1.025553,-0.615300,-0.441175,0.422739,-0.493584,-0.380604,-0.385704,-0.866731
3,0.485442,-1.034445,-0.863756,-0.928032,-0.939620,-0.939620,-1.076642,-0.885100,1.193709,-1.108424,...,-0.309266,-0.332742,1.487518,-0.209172,-0.318420,0.059261,-0.528292,0.844029,0.712632,-0.339636
4,0.954937,-0.559417,-0.639183,-0.209425,-0.678451,-0.678451,-0.834287,-0.025280,0.927480,-0.890283,...,-0.148391,-0.053983,-0.879385,-1.236134,-0.008419,0.176789,2.268667,-0.847641,-0.898070,0.840532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,0.174888,1.988638,1.860104,2.090119,1.804947,1.804947,2.428603,2.635953,-1.307443,2.180971,...,0.801149,1.672317,0.496886,1.075203,2.433291,-0.340854,-1.348389,5.253475,1.721990,1.428909
379,-0.811154,0.052940,0.005561,-0.509933,-0.104263,-0.104263,-0.109448,-0.373932,0.065413,-0.170708,...,-0.234774,-0.307686,-1.528489,-0.849166,-0.442280,-0.057591,0.420442,-0.683743,-0.971130,-0.072086
380,-1.055344,-0.063675,0.005561,-0.718983,-0.137233,-0.137233,-0.368478,-0.646472,0.317694,-0.422605,...,-0.336292,-0.450650,1.845521,2.565327,-0.557080,-0.540510,-0.858474,1.948538,2.622730,-0.272021
381,-0.801370,1.143929,1.048742,0.404658,0.928508,0.928508,1.321333,0.692292,-1.061236,1.290891,...,-0.210601,-0.233416,-2.003456,-0.802780,-0.236904,0.108385,0.440309,-0.633258,-0.960685,0.536345


In [24]:
RS_OS = -0.012222*X1[47] +0.085672*X1[62] -0.046874*X1[93]
RS_OS

0      0.118739
1      0.075784
2      0.115099
3      0.165825
4     -0.073798
         ...   
378    0.015628
379   -0.112108
380    0.192520
381   -0.157329
382    0.003363
Length: 383, dtype: float64

In [25]:
RS_OS.to_excel('RS_OS.xlsx', index = False)

In [28]:
RS_DFS = -0.015397*X1[17] +0.038338*X1[21] -0.002296*X1[44]-0.012222*X1[45]-0.104787*X1[47] -0.047772*X1[49] -0.020941*X1[80] +0.015440*X1[86] -0.023054*X1[93]
RS_DFS

0      0.201429
1      0.151346
2      0.168897
3      0.329350
4     -0.246623
         ...   
378   -0.054430
379   -0.042495
380    0.404509
381   -0.276831
382   -0.088235
Length: 383, dtype: float64

In [29]:
RS_DFS.to_excel('RS_DFS.xlsx', index = False)